In [1]:
import numpy as np
import symengine as si
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from torchkf import *

In [2]:
def make_layer(m, l, n, *args, **kwargs): 
    A  = np.random.randn(n, n)
    A  = (A - A.T) / (2 * 10)
    B  = np.random.randn(n, m)
    C  = np.random.randn(l, n)
    D  = np.random.randn(l, m)
    ia = np.arange(A.size)
    ib = np.arange(B.size) + ia[-1] + 1
    ic = np.arange(C.size) + ib[-1] + 1
    id = np.arange(D.size) + ic[-1] + 1
    
    pE = np.concatenate([A.flat, B.flat, C.flat, D.flat])
    
    def f(x, v, p): 
        x = x.reshape((n, 1))
        v = v.reshape((m, 1))
        p = p.reshape((-1,))
        A = p[ia].reshape((n, n))
        B = p[ib].reshape((n, m))
        return (A @ x).reshape((n, 1)) + (B @ v).reshape((n, 1))
    
    def g(x, v, p): 
        x = x.reshape((n, 1))
        v = v.reshape((m, 1))
        p = p.reshape((-1,))
        C = p[ic].reshape((l, n))
        D = p[id].reshape((l, m))
        return (C @ x).reshape((l, 1)) + (D @ v).reshape((l, 1))
    
    s = GaussianModel(
        n=n, l=l, m=m, 
        fsymb=f, gsymb=g, 
        pE=pE, pC=np.exp(32)
    )
    return s

In [3]:
model = HierarchicalGaussianModel(make_layer(3, 1, 5), make_layer(1, 3, 5), dt=0.1)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.07s)
  Compiling g... g() ok. (compiled in 0.02s)
Done. 
Compiling derivatives, it might take some time... 
  Compiling f... f() ok. (compiled in 0.04s)
  Compiling g... g() ok. (compiled in 0.01s)
Done. 


In [4]:
res = DEMInversion(model).generate(30)
# px.line(y=[*res.x[:, 0, :, 0].T])

  0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
model[0].d2f(model[0].x, model[1].v, model[0].pE).dx.dp.shape

(5, 5, 48)

In [6]:
model[1].p, model[1].pE.shape

(48, (48, 1))

In [7]:
t = np.linspace(0, 10, 100)
y = np.sin(2 * np.pi * 5 * t)[:, None]
DEMInversion(model).run(y, Emin=5)

E-step (F = -inf):   0%|          | 0/8 [00:00<?, ?it/s]

timestep:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
a, b = repeat(0, 2)

In [ ]:
%timeit np.zeros((5,0,0))
%timeit np.empty((5,0,0))

In [26]:
N = 5
M = 10
P = 3
A = np.arange(N*M*P).reshape((N, M, P))
B = A.swapaxes(0, 2)
A[0, 1, 2] = -128
A, B

(array([[[   0,    1,    2],
         [   3,    4, -128],
         [   6,    7,    8],
         [   9,   10,   11],
         [  12,   13,   14],
         [  15,   16,   17],
         [  18,   19,   20],
         [  21,   22,   23],
         [  24,   25,   26],
         [  27,   28,   29]],
 
        [[  30,   31,   32],
         [  33,   34,   35],
         [  36,   37,   38],
         [  39,   40,   41],
         [  42,   43,   44],
         [  45,   46,   47],
         [  48,   49,   50],
         [  51,   52,   53],
         [  54,   55,   56],
         [  57,   58,   59]],
 
        [[  60,   61,   62],
         [  63,   64,   65],
         [  66,   67,   68],
         [  69,   70,   71],
         [  72,   73,   74],
         [  75,   76,   77],
         [  78,   79,   80],
         [  81,   82,   83],
         [  84,   85,   86],
         [  87,   88,   89]],
 
        [[  90,   91,   92],
         [  93,   94,   95],
         [  96,   97,   98],
         [  99,  100,  101],
     

In [11]:
from itertools import *

np.fromiter(chain(*(np.arange(model[i].p) for i in range(2))), dtype='i')
np.fromiter(chain(*(model[i].p * [i] for i in range(2))), dtype='i')

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)